# 2) Galaxia 3-field run selection function

* [Check fileinfo for the Galaxia data](#check)
* [Generate selection function](#SF)
* [Calculate SF probabilities](#SFprob)

In [9]:
# autoreload modules when files have been altered
%load_ext autoreload
%autoreload 2

import os, sys, time
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

from seestar import ArrayMechanics
from seestar import SelectionGrid

import seaborn as sns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<a name="check"></a>
# Check file information

In [2]:
from seestar import surveyInfoPickler
fileinfo = surveyInfoPickler.surveyInformation('/media/andy/37E3-0F91/Research/SF/SFdata/Galaxia3_new/Galaxia3_new_fileinfo.pickle')
fileinfo.spectro_model = ('GMM', 1)
fileinfo.photo_model = ('GMM', 2)
#fileinfo.photo_coords = ['glon', 'glat', 'Japp', 'Kapp', 'Happ']
fileinfo.save()

<a name="SF"></a>
# Generate selection function

In [11]:
fileinfo_path = '/media/andy/37E3-0F91/Research/SF/SFdata/Galaxia3_new/Galaxia3_new_fileinfo.pickle'
SF = SelectionGrid.SFGenerator(fileinfo_path, ncores=3)
# ncores=0: SF will run the Serial version (not use multiprocessing)
# ncores>0: Parallel version on the number of cores given
# When running in parallel, the field counting can mess up a bit because the pools get disordered

Would you like the selection function in: a) observable, b) intrinsic, c) both? (return a, b or c)a
Path to observable SF (Galaxia3_new_obsSF.pickle) exists. Use this to ? (y/n)n
The spectro model description is:('GMM', 1)
The photo model description is:('GMM', 2)

{'glon': 'phi', 'Colmin': 'Colmin', 'glat': 'theta', 'Colmax': 'Colmax', 'Magmin': 'Magmin', 'Magmax': 'Magmax', 'fieldID': 'fieldID', 'halfangle': 'halfangle'}
Importing data for Colour-Magnitude Field interpolants...
{'Kapp': 'appB', 'Happ': 'appC', 'glon': 'phi', 'glat': 'theta', 'Japp': 'appA', 'fieldID': 'fieldID'}
Filtering for null values in spectro: Total star count = 5847. Filtered star count = 5847. 0 stars removed with null values
((8.001017701622784, 13.499959932502573), (0.5000188162888577, 1.1835355696129728))
...done.

Creating Colour-Magnitude Field interpolants...
multiprocessing - observable SF calculation - running on 3 cores...
Current field in col-mag calculation: 1.0, 1/3, Time: 1m, Left: 0m
now picklin

<a name="SFprob"></a>
# Calculate SF probabilities

In [ ]:
# Load in Isochrone Calculator
IsoCalculator = IsochroneScaling.IntrinsicToObservable()
IsoCalculator.LoadColMag(SF.isocolmag_pickle)

### Load in data

In [4]:
# Load in spectroscopic data
survey = pd.read_csv(fileinfo.spectro_path)
# Calculate colour and magnitude of spectroscopic data
survey['Colour'], survey['Happ'] = IsoCalculator.ColourMapp(survey.ageGyr, survey.mh, survey.mass, survey.s)

# Separate spectroscopic data into field 1 and field 2, 3 overlapping
survey1 = survey[survey[fileinfo.spectro_coords[0]] == 1.0]
survey23 = survey[survey[fileinfo.spectro_coords[0]].apply(lambda x: x in [2.0,3.0])]

NameError: name 'IsoCalculator' is not defined

In [7]:
# Load in field 1 photometric data

full1 = pd.read_csv(os.path.join(fileinfo.photo_path, str(1.0))+'.csv')
full1['Colour'] = full1[fileinfo.photo_coords[2]]-full1[fileinfo.photo_coords[3]]
full1.rename(index=str, columns={'feh':'mh', 'smass':'mass','rad':'s'}, inplace=True)
full1['Colour'], full1['Happ'] = IsoCalculator.ColourMapp(full1.ageGyr, full1.mh, full1.mass, full1.s)

In [9]:
# Load in field 2, 3 photometric data

full23 = pd.DataFrame()
fields= [2.0,3.0]
for field in fields:
    full23 = pd.concat((full23, pd.read_csv(os.path.join(fileinfo.photo_path, str(field))+'.csv')))
full23['Colour'] = full23[fileinfo.photo_coords[2]]-full23[fileinfo.photo_coords[3]]
full23.rename(index=str, columns={'feh':'mh', 'smass':'mass','rad':'s'}, inplace=True)
# Recalculate to make colour and apparent magnitude agree
full23['Colour'], full23['Happ'] = IsoCalculator.ColourMapp(full23.ageGyr, full23.mh, full23.mass, full23.s)

### Calculate SF probabilities

In [ ]:
# Calculate intrinsic SF probabilities for field 1
full1 = SF(full1, method='intrinsic', coords=['ageGyr', 'mh', 's', 'mass'], angle_coords=['glon', 'glat'])
full1['union_int'] = full1.union

In [ ]:
# Calculate observable SF probabilities for field 1
full1 = SF(full1, method='observable', coords=['Happ', 'Colour'], angle_coords=['glon', 'glat'])
full1['union_obs'] = full1.union

In [40]:
# Calculate intrinsic SF probabilities for field 2, 3
full23 = SF(full23, method='intrinsic', coords=['ageGyr', 'mh', 's', 'mass'], angle_coords=['glon', 'glat'])
full23['union_int'] = full23.union

Calculating all SF values...
0...0/14566...done
Calculating union contribution...
...done


In [10]:
# Calculate observable SF probabilities for field 2, 3
full23 = SF(full23, method='observable', coords=['Happ', 'Colour'], angle_coords=['glon', 'glat'])
full23['union_obs'] = full23.union

Calculating all SF values...
...14566/14566        ...done
Calculating union contribution...
...done


### Drop duplicates

In [15]:
# Clear duplicates for photometric field 2, 3
cleared = pd.DataFrame()
for i in range(len(full23)):
    result = np.sum((full23[i+1:].glon == full23.iloc[i].glon)&\
                    (full23[i+1:].glat == full23.iloc[i].glat))
    if result == 0: cleared = pd.concat((cleared, full23[i:i+1]))
    sys.stdout.write('\rDuplicate progress: %d / %d' % (i, len(full23)))

full23_c = cleared

Duplicate progress: 14565 / 14566

In [16]:
# Clear duplicates for spectroscopic fields 2,3
cleared = pd.DataFrame()
for i in range(len(survey23)):
    result = np.sum((survey23[i+1:].glon == survey23.iloc[i].glon)&\
                    (survey23[i+1:].glat == survey23.iloc[i].glat))
    if result == 0: cleared = pd.concat((cleared, survey23[i:i+1]))
    sys.stdout.write('\rDuplicate progress: %d / %d' % (i, len(survey23)))
    
survey23_c = cleared

Duplicate progress: 244 / 245

In [18]:
len(full23), len(full23_c), len(survey23), len(survey23_c)

(14566, 9536, 245, 239)

### Save files

In [21]:
# Save all data with selection function probabilities counted
full1.to_csv(os.path.join(fileinfo.photo_path, 'solutionF1.csv'), index=False)
full23.to_csv(os.path.join(fileinfo.photo_path, 'solutionF23.csv'), index=False)
survey1.to_csv(os.path.join(fileinfo.photo_path, 'solutionS1.csv'), index=False)
survey23.to_csv(os.path.join(fileinfo.photo_path, 'solutionS23.csv'), index=False)

full23_c.to_csv(os.path.join(fileinfo.photo_path, 'solutionF23_c.csv'), index=False)
survey23_c.to_csv(os.path.join(fileinfo.photo_path, 'solutionS23_c.csv'), index=False)